In [ ]:
import pandas as pd

In [ ]:
pass_data = pd.read_csv("/Users/annadaugaard/Desktop/VFF/VFF_analytics_src/data/03_model_data/test_for_streamit.csv")
gps_data=  pd.read_csv("/Users/annadaugaard/Desktop/VFF/preprocessed_data_for_decision_making.csv", index_col=0)

In [ ]:
gps_data_home = gps_data[gps_data["Team"]== "home"]

In [ ]:
def plot_trajectories_with_overlap(df, x_ball, y_ball, angle_degrees):
    import matplotlib.patches as patches
    import numpy as np
    import matplotlib.pyplot as plt
    from shapely.geometry import Polygon, Point

    # Filter home and away players
    home_players = df[df["Team"] == "home"]
    away_players = df[df["Team"] == "away"]

    # Ensure there are home players
    if home_players.empty:
        print("No home players found. Cannot plot trajectories.")
        return

    # Define team colors 
    team_colors = {"home": "blue", "away": "red"}

    # Loop over each home player and generate a separate plot
    for i, (_, target_player) in enumerate(home_players.iterrows()):
        # Compute trajectory angle
        dx = target_player["x"] - x_ball
        dy = target_player["y"] - y_ball
        trajectory_angle = np.arctan2(dy, dx)
        player_id = target_player["player_num"]

        # Compute distance (height of triangle)
        height = np.sqrt(dx**2 + dy**2)

        # Convert input angle from degrees to radians
        half_angle_radians = np.radians(angle_degrees / 2)

        # Compute base width using trigonometry
        base_width = 2 * height * np.tan(half_angle_radians)

        # Compute base corners at the player's location
        left_x = target_player["x"] + (base_width / 2) * np.cos(trajectory_angle + np.pi / 2)
        left_y = target_player["y"] + (base_width / 2) * np.sin(trajectory_angle + np.pi / 2)
        right_x = target_player["x"] + (base_width / 2) * np.cos(trajectory_angle - np.pi / 2)
        right_y = target_player["y"] + (base_width / 2) * np.sin(trajectory_angle - np.pi / 2)

        # Define the flipped triangle as a Shapely Polygon
        triangle = Polygon([(left_x, left_y), (right_x, right_y), (x_ball, y_ball)])

        # Calculate total overlap area with opponent circles
        total_overlap_area = 0

        for _, away_player in away_players.iterrows():
            # Define opponent's circle as a Shapely Point buffer
            opponent_circle = Point(away_player["x"], away_player["y"]).buffer(2)

            # Compute intersection area
            intersection = triangle.intersection(opponent_circle)
            total_overlap_area += intersection.area
            

        # Create the plot
        fig, ax = plt.subplots(figsize=(12, 8))

        # Plot players and their influence circles
        for _, row in df.iterrows():
            plt.scatter(row["x"], row["y"], color=team_colors[row["Team"]], alpha=0.7)
            circle = patches.Circle((row["x"], row["y"]), radius=4, color=team_colors[row["Team"]], alpha=0.3)
            ax.add_patch(circle)

        # Plot the ball position
        plt.scatter(x_ball, y_ball, color="red", label="Ball", s=100, edgecolor="black")

        # Plot dashed triangle outline (now flipped)
        flipped_triangle = patches.Polygon([(left_x, left_y), (right_x, right_y), (x_ball, y_ball)],
                                           edgecolor="black", facecolor="gray", linestyle="dashed", alpha=0.3)
        ax.add_patch(flipped_triangle)

        # Set limits for the coordinate system
        plt.xlim(0, 106)
        plt.ylim(0, 68)

        # Labels and legend
        plt.xlabel("X Coordinate")
        plt.ylabel("Y Coordinate")
        plt.title(f"Ball Trajectory to Home Player {player_id} | Overlap Area: {total_overlap_area:.2f}")
       #
       # plt.legend(["Home", "Away", "Ball", "Trajectory"])
        plt.grid(True)

        # Show the plot
        plt.show()

# Example usage
plot_trajectories_with_overlap(first_pass, x_ball=71.52, y_ball=37.05, angle_degrees=40)




In [ ]:
# Step 1: subset pass_data to home team only
pass_data_home = pass_data[pass_data["Team"] == "home"].reset_index(drop=True)
pass_data_home = pass_data_home[pass_data_home["uncertainty"] < 2].reset_index(drop=True)
# Columns to keep from gps_data
gps_columns = ["time", "player_id", "player_num", "x", "y", "spd", "ball_x","ball_y","Team","acceleration", "smoothed_acceleration"]

# Prepare an empty DataFrame for extracted data
extracted_gps_data = pd.DataFrame(columns=gps_columns + ["pass_event_id"])

# Steps 2 & 3: Extract GPS data for each pass event
for idx, row in pass_data_home.iterrows():
    from_player = row["From"]
    start_time = row["Start Time [s]"]
    end_time = row["End Time [s]"]
    
    # Extract relevant gps data
    gps_subset = gps_data_home[
        (gps_data_home["player_num"] == from_player) &
        (gps_data_home["time"] >= start_time) &
        (gps_data_home["time"] <= end_time)
    ][gps_columns].copy()

    # Add pass event identifier
    gps_subset["pass_event_id"] = idx
    
    # Concatenate results
    extracted_gps_data = pd.concat([extracted_gps_data, gps_subset], ignore_index=True)



In [ ]:
extracted_gps_data[extracted_gps_data['time'] == 96.72]

In [ ]:

# Calculate distance to the ball
extracted_gps_data["distance_to_ball"] = np.hypot(
    extracted_gps_data["x"] - extracted_gps_data["ball_x"],
    extracted_gps_data["y"] - extracted_gps_data["ball_y"]
)

# Plot
fig, ax1 = plt.subplots(figsize=(12, 6))

# Acceleration plot
color = 'tab:red'
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Acceleration', color=color)
ax1.plot(extracted_gps_data['time'][219:300], extracted_gps_data['acceleration'][219:300], color=color, marker='o', label='Acceleration')
ax1.tick_params(axis='y', labelcolor=color)

# Distance to ball plot on a second axis
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Distance to Ball (m)', color=color)
ax2.plot(extracted_gps_data['time'][219:300], extracted_gps_data['distance_to_ball'][219:300], color=color, marker='x', linestyle='--', label='Distance to Ball')
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.title("Acceleration and Distance to Ball Over Time")
plt.grid(True)
plt.show()



In [ ]:
extracted_gps_data[extracted_gps_data["time"] == 92.56]

In [ ]:
low_distance = extracted_gps_data[extracted_gps_data["distance_to_ball"] < 2]


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
df = low_distance.copy()

# Identify chunks by consecutive player_num
df['player_chunk'] = (df['player_num'] != df['player_num'].shift()).cumsum()

# Get index of max acceleration per chunk
max_acceleration_indices = df.groupby('player_chunk')['acceleration'].idxmax() - 10

# Initialize a list of zeros with length 2500
binary_list = [0] * len(extracted_gps_data)

In [ ]:

# Set positions from max_acceleration_indices to 1, ensuring indices within bounds
for idx in max_acceleration_indices:
    if 0 <= idx < len(extracted_gps_data):
        binary_list[idx] = 1

In [ ]:

# Verify the result (optional)
extracted_gps_data["decision_making_point"] = binary_list


In [ ]:
plt.plot(extracted_gps_data["decision_making_point"] )

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point

# Robust overlap area calculation function
def calculate_overlap_area(target_x, target_y, x_ball, y_ball, away_players, angle_degrees=40, circle_radius=2):
    dx, dy = target_x - x_ball, target_y - y_ball
    height = np.hypot(dx, dy)
    if height == 0:
        return 0.0

    trajectory_angle = np.arctan2(dy, dx)
    half_angle_radians = np.radians(angle_degrees / 2)
    base_width = 2 * height * np.tan(half_angle_radians)

    left_x = target_x + (base_width / 2) * np.cos(trajectory_angle + np.pi / 2)
    left_y = target_y + (base_width / 2) * np.sin(trajectory_angle + np.pi / 2)
    right_x = target_x + (base_width / 2) * np.cos(trajectory_angle - np.pi / 2)
    right_y = target_y + (base_width / 2) * np.sin(trajectory_angle - np.pi / 2)

    points = [(left_x, left_y), (right_x, right_y), (x_ball, y_ball)]

    triangle = Polygon(points)
    if not triangle.is_valid or triangle.area == 0:
        return 0.0

    total_overlap_area = sum(
        triangle.intersection(Point(away_x, away_y).buffer(circle_radius)).area
        for away_x, away_y in zip(away_players["x"], away_players["y"])
    )

    return total_overlap_area

# Updated custom scoring function with directional boost
def custom_score(overlap_area, distance_to_ball, ball_direction_x, alpha=1.0, beta=0.01, gamma=0.1):
    direction_bonus = gamma if ball_direction_x > 0 else -gamma
    penalty = alpha * overlap_area + beta * distance_to_ball
    return np.exp(-penalty + direction_bonus)

# Placeholder for results
results = []

# Iterate over each row in extracted_gps_data with decision-making points
decision_points = extracted_gps_data[extracted_gps_data["decision_making_point"] == 1].reset_index()

for idx, row in decision_points.iterrows():
    current_time = row["time"]
    current_player_num = row["player_num"]
    x_ball, y_ball = row["ball_x"], row["ball_y"]

    players_at_time = gps_data[(gps_data["time"] == current_time) & 
                               (gps_data["player_num"] != current_player_num)]

    home_players = players_at_time[players_at_time["Team"] == "home"]
    away_players = players_at_time[players_at_time["Team"] == "away"]

    for _, home_player in home_players.iterrows():
        
        ball_direction_x = x_ball -home_player["x"] 

        distance_to_ball = np.hypot(home_player["x"] - x_ball, home_player["y"] - y_ball)
        
        if distance_to_ball < 15:
            circle_radius = 2
        elif 15 < distance_to_ball <= 25:
            circle_radius =3
        else:
            circle_radius=6
        
        overlap_area = calculate_overlap_area(
                target_x=home_player["x"],
                target_y=home_player["y"],
                x_ball=x_ball,
                y_ball=y_ball,
                away_players=away_players,
                angle_degrees=40,
                circle_radius=circle_radius
            )
        # Using custom scoring with directional boost
        score = custom_score(overlap_area, distance_to_ball, ball_direction_x)

        results.append({
            "timestamp": current_time,
            "reference_player_num": current_player_num,
            "target_player_num": home_player["player_num"],
            "score": score,
            "pass_event_id":row["pass_event_id"]
        })

results_df = pd.DataFrame(results)

# Convert to wide format
results_df_wide = results_df.pivot_table(
    index=['timestamp', "pass_event_id",'reference_player_num'],
    columns='target_player_num',
    values='score'
).reset_index()

results_df_wide.columns = [
    f'score_to_player_{int(col)}' if isinstance(col, (int, float)) else col
    for col in results_df_wide.columns
]

# Select columns containing 'score_to_player'
score_columns = [col for col in results_df_wide.columns if 'score_to_player' in col]

# Find the max value per row among score columns
results_df_wide['max_score'] = results_df_wide[score_columns].max(axis=1)

# Find the corresponding player column for each max score
results_df_wide['max_score_player'] = results_df_wide[score_columns].idxmax(axis=1)

# Display the results
print(results_df_wide[['timestamp', 'reference_player_num', 'max_score', 'max_score_player']])

# Extract just the numeric player ID
results_df_wide['max_score_player'] = results_df_wide['max_score_player'].str.extract('score_to_player_(\d+)').astype(int)


In [ ]:
results_df_wide

In [ ]:
import matplotlib.patches as patches
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point

def plot_event_overlap(timestamp, from_player_num, gps_data, angle_degrees=40, circle_radius=2):
    # Extract positions at the given timestamp
    df_timestamp = gps_data[gps_data["time"] == timestamp]

    if df_timestamp.empty:
        print(f"No data available for timestamp {timestamp}")
        return

    # Separate home and away teams
    home_players = df_timestamp[df_timestamp["Team"] == "home"]
    away_players = df_timestamp[df_timestamp["Team"] == "away"]

    # Extract the "from" player's position and ball position
    from_player = df_timestamp[df_timestamp["player_num"] == from_player_num].iloc[0]
    x_ball, y_ball = from_player["ball_x"], from_player["ball_y"]

    # Define team colors
    team_colors = {"home": "blue", "away": "red"}

    # Plot setup
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot all players with their influence circles
    for _, row in df_timestamp.iterrows():
        ax.scatter(row["x"], row["y"], color=team_colors[row["Team"]], alpha=0.7)
        circle = patches.Circle((row["x"], row["y"]), radius=4, color=team_colors[row["Team"]], alpha=0.3)
        ax.add_patch(circle)

    # Highlight the ball
    ax.scatter(x_ball, y_ball, color="yellow", label="Ball", s=120, edgecolor="black", zorder=5)

    # Plot triangle areas for home players (excluding the "from" player itself)
    for _, target_player in home_players.iterrows():
        player_num = target_player["player_num"]
        if player_num == from_player_num:
            continue

        dx, dy = target_player["x"] - x_ball, target_player["y"] - y_ball
        height = np.hypot(dx, dy)
        if height == 0:
            continue

        trajectory_angle = np.arctan2(dy, dx)
        half_angle_radians = np.radians(angle_degrees / 2)
        base_width = 2 * height * np.tan(half_angle_radians)

        left_x = target_player["x"] + (base_width / 2) * np.cos(trajectory_angle + np.pi / 2)
        left_y = target_player["y"] + (base_width / 2) * np.sin(trajectory_angle + np.pi / 2)
        right_x = target_player["x"] + (base_width / 2) * np.cos(trajectory_angle - np.pi / 2)
        right_y = target_player["y"] + (base_width / 2) * np.sin(trajectory_angle - np.pi / 2)

        triangle = Polygon([(left_x, left_y), (right_x, right_y), (x_ball, y_ball)])

        total_overlap_area = sum(
            triangle.intersection(Point(away_x, away_y).buffer(circle_radius)).area
            for away_x, away_y in zip(away_players["x"], away_players["y"])
        )

        triangle_patch = patches.Polygon(
            [(left_x, left_y), (right_x, right_y), (x_ball, y_ball)],
            edgecolor="black", facecolor="gray", linestyle="dashed", alpha=0.3
        )
        ax.add_patch(triangle_patch)

        ax.text(target_player["x"], target_player["y"], f'{player_num}\n({total_overlap_area:.1f})', 
                fontsize=10, ha='center', va='center', color='black')

    # Set limits and labels
    ax.set_xlim(0, 106)
    ax.set_ylim(0, 68)
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.set_title(f"Player positions and overlap at timestamp {timestamp} | From Player: {from_player_num}")
    plt.grid(True)

    plt.show()

# Example call:
# plot_event_overlap(timestamp=1.48, from_player_num=24, gps_data=gps_data, angle_degrees=40)


In [ ]:
import matplotlib.patches as patches
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point

def plot_event_overlap(timestamp, from_player_num, gps_data, angle_degrees=40):
    # Extract positions at the given timestamp
    df_timestamp = gps_data[gps_data["time"] == timestamp]

    if df_timestamp.empty:
        print(f"No data available for timestamp {timestamp}")
        return

    # Separate home and away teams
    home_players = df_timestamp[df_timestamp["Team"] == "home"]
    away_players = df_timestamp[df_timestamp["Team"] == "away"]

    # Extract the "from" player's position and ball position
    from_player = df_timestamp[df_timestamp["player_num"] == from_player_num].iloc[0]
    x_ball, y_ball = from_player["ball_x"], from_player["ball_y"]

    # Define team colors
    team_colors = {"home": "blue", "away": "red"}

    # Plot setup
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot all players with their dynamic influence circles
    for _, row in df_timestamp.iterrows():
        distance_to_ball = np.hypot(row["x"] - x_ball, row["y"] - y_ball)

        if distance_to_ball < 15:
            circle_radius = 2
        elif 15 < distance_to_ball <= 25:
            circle_radius = 3
        else:
            circle_radius = 6

        ax.scatter(row["x"], row["y"], color=team_colors[row["Team"]], alpha=0.7)
        circle = patches.Circle((row["x"], row["y"]), radius=circle_radius, color=team_colors[row["Team"]], alpha=0.3)
        ax.add_patch(circle)

    # Highlight the ball
    ax.scatter(x_ball, y_ball, color="yellow", label="Ball", s=120, edgecolor="black", zorder=5)

    # Plot triangle areas for home players (excluding the "from" player itself)
    for _, target_player in home_players.iterrows():
        player_num = target_player["player_num"]
        if player_num == from_player_num:
            continue

        distance_to_ball = np.hypot(target_player["x"] - x_ball, target_player["y"] - y_ball)

        if distance_to_ball < 15:
            circle_radius = 2
        elif 15 < distance_to_ball <= 25:
            circle_radius = 3
        else:
            circle_radius = 6

        dx, dy = target_player["x"] - x_ball, target_player["y"] - y_ball
        height = np.hypot(dx, dy)
        if height == 0:
            continue

        trajectory_angle = np.arctan2(dy, dx)
        half_angle_radians = np.radians(angle_degrees / 2)
        base_width = 2 * height * np.tan(half_angle_radians)

        left_x = target_player["x"] + (base_width / 2) * np.cos(trajectory_angle + np.pi / 2)
        left_y = target_player["y"] + (base_width / 2) * np.sin(trajectory_angle + np.pi / 2)
        right_x = target_player["x"] + (base_width / 2) * np.cos(trajectory_angle - np.pi / 2)
        right_y = target_player["y"] + (base_width / 2) * np.sin(trajectory_angle - np.pi / 2)

        triangle = Polygon([(left_x, left_y), (right_x, right_y), (x_ball, y_ball)])

        total_overlap_area = sum(
            triangle.intersection(Point(away_x, away_y).buffer(circle_radius)).area
            for away_x, away_y in zip(away_players["x"], away_players["y"])
        )

        triangle_patch = patches.Polygon(
            [(left_x, left_y), (right_x, right_y), (x_ball, y_ball)],
            edgecolor="black", facecolor="gray", linestyle="dashed", alpha=0.3
        )
        ax.add_patch(triangle_patch)

        ax.text(target_player["x"], target_player["y"], f'{player_num}\n({total_overlap_area:.1f})', 
                fontsize=10, ha='center', va='center', color='black')

    # Set limits and labels
    ax.set_xlim(0, 106)
    ax.set_ylim(0, 68)
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.set_title(f"Player positions and overlap at timestamp {timestamp} | From Player: {from_player_num}")
    plt.grid(True)

    plt.show()

In [ ]:
only_24 = results_df_wide[results_df_wide["reference_player_num"] == 24]
only_2 = results_df_wide[results_df_wide["reference_player_num"] == 2]

In [ ]:
only_24

In [ ]:
only_24_reset_index= only_24.reset_index(drop=True)
only_24_reset_index.index= only_24_reset_index.index + 1

In [ ]:
from pathlib import Path
# Folder containing the plot images
folder = Path("/Users/annadaugaard/Desktop/VFF/explore/plots_annotations_payer_24_focus")

# Extract plot IDs from filenames
plot_ids = []
pattern = r"plot_(\d+)_from_\d+_to_\d+\.png"

for file in folder.glob("plot_*.png"):
    match = re.match(pattern, file.name)
    if match:
        plot_ids.append(int(match.group(1)))

# Now subset the dataframe by index
subset_df = only_24_reset_index[only_24_reset_index.index.isin(plot_ids)]

In [ ]:
subset_df.to_csv("passes_from_player24_plots_and_pass_events.csv")

In [ ]:
len(subset_df)

In [ ]:
plot_event_overlap(2547.84, 23, gps_data, angle_degrees=40)

In [ ]:
plot_event_overlap(99, 16, gps_data, angle_degrees=40)

In [ ]:
import matplotlib.patches as patches
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point

def plot_event_overlap(timestamp, from_player_num, gps_data, optimal_player_num, angle_degrees=40):
    # Extract positions at the given timestamp
    df_timestamp = gps_data[gps_data["time"] == timestamp]

    if df_timestamp.empty:
        print(f"No data available for timestamp {timestamp}")
        return

    # Separate home and away teams
    home_players = df_timestamp[df_timestamp["Team"] == "home"]
    away_players = df_timestamp[df_timestamp["Team"] == "away"]

    # Extract the "from" player's position and ball position
    from_player = df_timestamp[df_timestamp["player_num"] == from_player_num].iloc[0]
    x_ball, y_ball = from_player["ball_x"], from_player["ball_y"]

    # Define team colors
    team_colors = {"home": "blue", "away": "red"}

    # Plot setup
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot all players with their dynamic influence circles
    for _, row in df_timestamp.iterrows():
        distance_to_ball = np.hypot(row["x"] - x_ball, row["y"] - y_ball)

        if distance_to_ball < 15:
            circle_radius = 2
        elif 15 < distance_to_ball <= 25:
            circle_radius = 3
        else:
            circle_radius = 6

        ax.scatter(row["x"], row["y"], color=team_colors[row["Team"]], alpha=0.7)
        circle = patches.Circle((row["x"], row["y"]), radius=circle_radius, color=team_colors[row["Team"]], alpha=0.3)
        ax.add_patch(circle)

    # Highlight the ball
    ax.scatter(x_ball, y_ball, color="yellow", label="Ball", s=120, edgecolor="black", zorder=5)

    # Plot only the optimal player's triangle area
    optimal_player = home_players[home_players["player_num"] == optimal_player_num].iloc[0]

    dx, dy = optimal_player["x"] - x_ball, optimal_player["y"] - y_ball
    height = np.hypot(dx, dy)
    if height != 0:
        trajectory_angle = np.arctan2(dy, dx)
        half_angle_radians = np.radians(angle_degrees / 2)
        base_width = 2 * height * np.tan(half_angle_radians)

        left_x = optimal_player["x"] + (base_width / 2) * np.cos(trajectory_angle + np.pi / 2)
        left_y = optimal_player["y"] + (base_width / 2) * np.sin(trajectory_angle + np.pi / 2)
        right_x = optimal_player["x"] + (base_width / 2) * np.cos(trajectory_angle - np.pi / 2)
        right_y = optimal_player["y"] + (base_width / 2) * np.sin(trajectory_angle - np.pi / 2)

        triangle_patch = patches.Polygon(
            [(left_x, left_y), (right_x, right_y), (x_ball, y_ball)],
            edgecolor="green", facecolor="green", linestyle="dashed", alpha=0.3
        )
        ax.add_patch(triangle_patch)

        ax.text(optimal_player["x"], optimal_player["y"], f'{int(optimal_player_num)}', 
                fontsize=12, ha='center', va='center', color='black', weight='bold')

    # Set limits and labels
    ax.set_xlim(0, 106)
    ax.set_ylim(0, 68)
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.set_title(f"Optimal pass option at timestamp {timestamp} | From Player: {int(from_player_num)} to Player: {int(optimal_player_num)}")
    plt.grid(True)

In [ ]:
import os

# Create the output directory if it doesn't exist
output_folder = "plots2"
os.makedirs(output_folder, exist_ok=True)

# Loop and save plots
for idx, row in only_24.iterrows():
    timestamp = row["timestamp"]
    from_player_num = row["reference_player_num"]
    optimal_player_num = row["max_score_player"]

    # Plot the event
    plot_event_overlap(timestamp, from_player_num, gps_data, optimal_player_num,angle_degrees=40)

    # Save the plot to file
    plt.savefig(os.path.join(output_folder, f"plot_{idx}_ts_{timestamp}_from_{int(from_player_num)}_to_{int(optimal_player_num)}.png"))
    plt.close()  # Close the figure to avoid memory issues


In [ ]:
for idx, row in results_df_wide[0:20].iterrows():
    
    plot_event_overlap(results_df_wide["timestamp"][idx], results_df_wide["reference_player_num"][idx], gps_data, results_df_wide['max_score_player'][idx], angle_degrees=40, circle_radius=2)

In [ ]:

plot_event_overlap(results_df_wide["timestamp"][0], results_df_wide["reference_player_num"][0], gps_data, results_df_wide['max_score_player'][0])

In [ ]:
import matplotlib.patches as patches
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point

def plot_event_overlap(timestamp, from_player_num, gps_data, optimal_player_num, pass_data, angle_degrees=40, circle_radius=2):
    # Extract positions at the given timestamp
    df_timestamp = gps_data[gps_data["time"] == timestamp]

    if df_timestamp.empty:
        print(f"No data available for timestamp {timestamp}")
        return

    # Separate home and away teams
    home_players = df_timestamp[df_timestamp["Team"] == "home"]
    away_players = df_timestamp[df_timestamp["Team"] == "away"]

    # Extract the "from" player's position and ball position
    from_player = df_timestamp[df_timestamp["player_num"] == from_player_num].iloc[0]
    x_ball, y_ball = from_player["ball_x"], from_player["ball_y"]

    # Find actual pass recipient from pass_data
    actual_pass = pass_data[(pass_data["From"] == from_player_num) &
                            (pass_data["Start Time [s]"] <= timestamp) &
                            (pass_data["End Time [s]"] >= timestamp)]

    actual_pass_player_num = actual_pass["To"].iloc[0] if not actual_pass.empty else None

    # Define team colors
    team_colors = {"home": "blue", "away": "red"}

    # Plot setup
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot all players with their influence circles
    for _, row in df_timestamp.iterrows():
        ax.scatter(row["x"], row["y"], color=team_colors[row["Team"]], alpha=0.7)
        circle = patches.Circle((row["x"], row["y"]), radius=4, color=team_colors[row["Team"]], alpha=0.3)
        ax.add_patch(circle)

    # Highlight the ball
    ax.scatter(x_ball, y_ball, color="yellow", label="Ball", s=120, edgecolor="black", zorder=5)

    # Highlight optimal player's triangle area
    optimal_player = home_players[home_players["player_num"] == optimal_player_num].iloc[0]

    dx, dy = optimal_player["x"] - x_ball, optimal_player["y"] - y_ball
    height = np.hypot(dx, dy)
    if height != 0:
        trajectory_angle = np.arctan2(dy, dx)
        half_angle_radians = np.radians(angle_degrees / 2)
        base_width = 2 * height * np.tan(half_angle_radians)

        left_x = optimal_player["x"] + (base_width / 2) * np.cos(trajectory_angle + np.pi / 2)
        left_y = optimal_player["y"] + (base_width / 2) * np.sin(trajectory_angle + np.pi / 2)
        right_x = optimal_player["x"] + (base_width / 2) * np.cos(trajectory_angle - np.pi / 2)
        right_y = optimal_player["y"] + (base_width / 2) * np.sin(trajectory_angle - np.pi / 2)

        triangle_patch = patches.Polygon(
            [(left_x, left_y), (right_x, right_y), (x_ball, y_ball)],
            edgecolor="green", facecolor="green", linestyle="dashed", alpha=0.3
        )
        ax.add_patch(triangle_patch)

        ax.text(optimal_player["x"], optimal_player["y"], f'{optimal_player_num}', 
                fontsize=12, ha='center', va='center', color='green', weight='bold')

    # Highlight actual pass recipient
    if actual_pass_player_num and actual_pass_player_num != optimal_player_num:
        actual_player = home_players[home_players["player_num"] == actual_pass_player_num]
        if not actual_player.empty:
            actual_player = actual_player.iloc[0]
            ax.scatter(actual_player["x"], actual_player["y"], color="purple", s=150, edgecolor="black", zorder=6)
            ax.text(actual_player["x"], actual_player["y"], f'{actual_pass_player_num}\n(actual)',
                    fontsize=12, ha='center', va='center', color='purple', weight='bold')

    # Set limits and labels
    ax.set_xlim(0, 106)
    ax.set_ylim(0, 68)
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.set_title(f"Optimal pass at {timestamp}s | From Player: {from_player_num} to Player: {optimal_player_num}")
    plt.grid(True)

    #plt.show()

# Example call:
# 

#results_df_wide["timestamp"][0], results_df_wide["reference_player_num"][0], gps_data, results_df_wide['max_score_player'][0], angle_degrees=40, circle_radius=2)

In [ ]:
import matplotlib.patches as patches
import matplotlib.patheffects as path_effects
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point

def plot_event_overlap(timestamp, from_player_num, gps_data, optimal_player_num, pass_data, angle_degrees=40):
    # Extract positions at the given timestamp
    df_timestamp = gps_data[gps_data["time"] == timestamp]

    if df_timestamp.empty:
        print(f"No data available for timestamp {timestamp}")
        return

    # Separate home and away teams
    home_players = df_timestamp[df_timestamp["Team"] == "home"]
    away_players = df_timestamp[df_timestamp["Team"] == "away"]

    # Extract the "from" player's position and ball position
    from_player = df_timestamp[df_timestamp["player_num"] == from_player_num].iloc[0]
    x_ball, y_ball = from_player["ball_x"], from_player["ball_y"]

    # Find actual pass recipient from pass_data
    actual_pass = pass_data[(pass_data["From"] == from_player_num) &
                            (pass_data["Start Time [s]"] <= timestamp) &
                            (pass_data["End Time [s]"] >= timestamp)]

    actual_pass_player_num = actual_pass["To"].iloc[0] if not actual_pass.empty else None

    # Define team colors
    team_colors = {"home": "blue", "away": "red"}

    # Plot setup
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot all players with dynamic influence circles and numbers
    for _, row in df_timestamp.iterrows():
        distance_to_ball = np.hypot(row["x"] - x_ball, row["y"] - y_ball)

        if distance_to_ball < 15:
            circle_radius = 2
        elif 15 < distance_to_ball <= 25:
            circle_radius = 3
        else:
            circle_radius = 6

        ax.scatter(row["x"], row["y"], color=team_colors[row["Team"]], alpha=0.7)
        circle = patches.Circle((row["x"], row["y"]), radius=circle_radius, color=team_colors[row["Team"]], alpha=0.3)
        ax.add_patch(circle)

        ax.text(row["x"], row["y"], f'{row["player_num"]}', fontsize=10, ha='center', va='center',
                color='black', weight='bold', path_effects=[path_effects.withStroke(linewidth=3, foreground="white")])

    # Highlight the ball
    ax.scatter(x_ball, y_ball, color="yellow", label="Ball", s=120, edgecolor="black", zorder=5)

    # Highlight optimal player's triangle area
    optimal_player = home_players[home_players["player_num"] == optimal_player_num].iloc[0]

    dx, dy = optimal_player["x"] - x_ball, optimal_player["y"] - y_ball
    height = np.hypot(dx, dy)
    if height != 0:
        trajectory_angle = np.arctan2(dy, dx)
        half_angle_radians = np.radians(angle_degrees / 2)
        base_width = 2 * height * np.tan(half_angle_radians)

        left_x = optimal_player["x"] + (base_width / 2) * np.cos(trajectory_angle + np.pi / 2)
        left_y = optimal_player["y"] + (base_width / 2) * np.sin(trajectory_angle + np.pi / 2)
        right_x = optimal_player["x"] + (base_width / 2) * np.cos(trajectory_angle - np.pi / 2)
        right_y = optimal_player["y"] + (base_width / 2) * np.sin(trajectory_angle - np.pi / 2)

        triangle_patch = patches.Polygon(
            [(left_x, left_y), (right_x, right_y), (x_ball, y_ball)],
            edgecolor="green", facecolor="green", linestyle="dashed", alpha=0.3
        )
        ax.add_patch(triangle_patch)

    # Highlight actual pass recipient
    if actual_pass_player_num and actual_pass_player_num != optimal_player_num:
        actual_player = home_players[home_players["player_num"] == actual_pass_player_num]
        if not actual_player.empty:
            actual_player = actual_player.iloc[0]
            ax.scatter(actual_player["x"], actual_player["y"], color="purple", s=150, edgecolor="black", zorder=6)
            ax.text(actual_player["x"], actual_player["y"], f'{actual_pass_player_num}\n(actual)',
                    fontsize=12, ha='center', va='center', color='purple', weight='bold',
                    path_effects=[path_effects.withStroke(linewidth=3, foreground="white")])

    # Set limits and labels
    ax.set_xlim(0, 106)
    ax.set_ylim(0, 68)
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.set_title(f"Optimal pass at {timestamp}s | From Player: {from_player_num} to Player: {optimal_player_num}")
    plt.grid(True)

# plot_event_overlap(timestamp=1.48, from_player_num=24, optimal_player_num=12, gps_data=gps_data, pass_data=pass_data, angle_degrees=40)

# Example call:
# plot_event_overlap(timestamp=1.48, from_player_num=24, optimal_player_num=12, gps_data=gps_data, pass_data=pass_data, angle_degrees=40)


In [ ]:
import matplotlib.patches as patches
import matplotlib.patheffects as path_effects
import numpy as np
import matplotlib.pyplot as plt

def plot_players_and_ball(timestamp, gps_data):
    # Extract positions at the given timestamp
    df_timestamp = gps_data[gps_data["time"] == timestamp]

    if df_timestamp.empty:
        print(f"No data available for timestamp {timestamp}")
        return

    # Define team colors
    team_colors = {"home": "blue", "away": "red"}

    # Plot setup
    fig, ax = plt.subplots(figsize=(10, 7))
    

    # Find ball position (assuming it's stored with a player)
    ball_row = df_timestamp.iloc[0]
    x_ball, y_ball = ball_row["ball_x"], ball_row["ball_y"]

    # Plot all players with influence circles
    for _, row in df_timestamp.iterrows():
        distance_to_ball = np.hypot(row["x"] - x_ball, row["y"] - y_ball)

        if distance_to_ball < 15:
            circle_radius = 2
        elif 15 < distance_to_ball <= 25:
            circle_radius = 3
        else:
            circle_radius = 6

        ax.scatter(row["x"], row["y"], color=team_colors[row["Team"]], alpha=0.7)
        circle = patches.Circle((row["x"], row["y"]), radius=circle_radius,
                                color=team_colors[row["Team"]], alpha=0.3)
        ax.add_patch(circle)

        ax.text(row["x"], row["y"], f'{row["player_num"]}', fontsize=10,
                ha='center', va='center', color='black', weight='bold',
                path_effects=[path_effects.withStroke(linewidth=3, foreground="white")])

    # Plot ball
    ax.scatter(x_ball, y_ball, color="yellow", label="Ball", s=120,
               edgecolor="black", zorder=5)

    # Set limits and labels
    ax.set_xlim(0, 110)
    ax.set_ylim(0, 72)
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    ax.set_title(f"Players and Ball Positions at {timestamp}s")
    plt.grid(True)
    #plt.show()

In [ ]:
# Create the output directory if it doesn't exist

output_folder = "plots_annotations_only_player_44"
os.makedirs(output_folder, exist_ok=True)
count = 0
for idx, row in only_2.iterrows():
    count += 1
    #plot_event_overlap(timestamp=row["timestamp"], from_player_num=row["reference_player_num"], optimal_player_num=row["max_score_player"], gps_data=gps_data, pass_data=pass_data, angle_degrees=40)
        # Save the plot to file
    plot_players_and_ball(timestamp=row["timestamp"], gps_data=gps_data)
    plt.savefig(os.path.join(output_folder, f"plot_{count}_from_{int(row["reference_player_num"])}_to_{int(row["max_score_player"])}.png"))
    plt.close()  # Close the figure to avoid memory issues

In [ ]:
import os
import shutil
import re

source_folder = '/Users/annadaugaard/Desktop/VFF/explore/plots_annotations_only_player_2'
dest_folder = '/Users/annadaugaard/Desktop/VFF/explore/plots_til_annotering_player_2'

# Create destination folder if it doesn't exist
os.makedirs(dest_folder, exist_ok=True)

# Regex pattern to extract the number after "plot_"
pattern = re.compile(r'(plot_\d+)_from.*')

for filename in os.listdir(source_folder):
    match = pattern.match(filename)
    if match:
        new_filename = f"{match.group(1)}.png"  # Adjust extension if needed
        src = os.path.join(source_folder, filename)
        
        dst = os.path.join(dest_folder, new_filename)
        shutil.copy(src, dst)
        print(f"Copied {filename} to {new_filename}")



In [ ]:
image_files

In [ ]:
import os
import re
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from PIL import Image as PILImage

# Folder containing images
image_folder = "/Users/annadaugaard/Desktop/VFF/explore/spiller_24_fokus"
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
image_files = sorted(image_files, key=lambda x: int(x.split('_')[1].split('.')[0]))

# Create workbook and sheet
wb = Workbook()
ws = wb.active
ws.title = "Plots"

# Set column headers
ws.cell(row=1, column=1, value="Picture ID")
ws.cell(row=1, column=2, value="Image")
ws.cell(row=1, column=3, value="Choice")

# Adjust columns width
ws.column_dimensions['A'].width = 15
ws.column_dimensions['B'].width = 50
ws.column_dimensions['C'].width = 20

# Insert images with their IDs
for idx, img_file in enumerate(image_files, start=2):  # start from row 2 due to headers
    # Extract numeric ID from filename (e.g., plot_1.png -> 1)
    match = re.search(r'(\d+)', img_file)
    pic_id = match.group(1) if match else "Unknown"

    # Add Picture ID in column A
    ws.cell(row=idx, column=1, value=int(pic_id))

    # Prepare image path
    img_path = os.path.join(image_folder, img_file)
    img = PILImage.open(img_path)
    
    max_height = 600
    if img.height > max_height:
        ratio = max_height / float(img.height)
        new_width = int(img.width * ratio)
        img = img.resize((new_width, max_height), PILImage.LANCZOS)
        resized_path = os.path.join(image_folder, f"resized_{img_file}")
        img.save(resized_path)
        img_to_excel = Image(resized_path)
    else:
        img_to_excel = Image(img_path)

    # Insert image in column B
    cell_location = f"B{idx}"
    ws.row_dimensions[idx].height = img_to_excel.height * 0.75
    ws.add_image(img_to_excel, cell_location)

    # Column C ("Choice") remains blank for user input
    ws.cell(row=idx, column=3, value="")

# Save workbook
wb.save("plots_with_images_spiller_24.xlsx")
print("Excel file created successfully!")
